In [1]:
#import necessary Libraries
%matplotlib inline
import os
import numpy as np
import random
import tensorflow as tf
import cv2
import keras
from keras.utils import np_utils
from tensorflow.keras.optimizers import RMSprop, SGD
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D,BatchNormalization, MaxPooling2D
from keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from skimage import io
import os
import scipy.misc
from keras.preprocessing import image
from imageio import imread
from keras import regularizers
import csv
from keras.models import Sequential

In [2]:

datasets_path = '/content/drive/MyDrive/Dataset/101_ObjectCategories' #Add the path to the dataset

def load_images(path,n=0): 
    X = [] # intializae array
    Y=[]
    i=-1
    labels = []
    for label in os.listdir(path): #take folder name as a lable of imgaes froo given path's list of directory
        back_path = os.path.join(path,label)
        labels.append(label)  # stores lable in lables named array
        i = i+1
        for filename in os.listdir(back_path):  # for loop to take images from each folder from dataset
            image_path = os.path.join(back_path,filename)  # image path
            img = image.load_img(image_path,target_size=(128,128)) # using keras load_img function load image with size of 128*128
            img = image.img_to_array(img) # convert image to array using keras img_to_array function it will give array 128*128*3
            img[:,:,0] -= 123.68
            img[:,:,1] -= 116.78
            img[:,:,2] -= 103.94
            
            Y.append(i) # append lable count  in Y array
            X.append(img) # appand converted image to araray in X array
            
    return X,Y,labels

x_train,y,label_data = load_images(datasets_path) # call load_image function and store return value 
X = np.array(x_train) # convert into numpy array

Y = np.array(y)
print(X.shape)

(9145, 128, 128, 3)


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25) # split dataset into train and test using train_test_split function

In [4]:
number_of_classes = 102 # number of classes in dataset
Y_train = np_utils.to_categorical(y_train)
Y_test = np_utils.to_categorical(y_test)

In [5]:
model = Sequential()
# 1st layer..........
model.add(Conv2D(96, kernel_size = (5, 5), strides = (2,2), activation='relu', padding = 'valid' , data_format='channels_last', input_shape=(128, 128, 3),name = "first"))
model.add(MaxPooling2D(pool_size=(3,3) , strides = (2,2) , padding = 'valid' )) 
model.add(BatchNormalization())

# 2nd layer................
model.add(Conv2D(256, kernel_size=(3,3), activation='relu', strides=(1,1), padding = 'valid', name = "second"))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
model.add(BatchNormalization())

#3rd layer.................
model.add(Conv2D(384, kernel_size=(3,3), activation='relu', strides=(1,1), padding='valid', name = "third"))
model.add(BatchNormalization())

#4th layer.................
model.add(Conv2D(384, kernel_size=(3,3), activation='relu', strides=(1,1), padding='valid', name = "fourth"))
model.add(BatchNormalization())

#5th layer.................
model.add(Conv2D(256, kernel_size=(3,3), strides=(1,1), activation = 'relu', padding='valid' , name = "fifth"))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
model.add(BatchNormalization())

model.add(Dropout(0.2))

model.add(Flatten())

# Fully Connected Layer 1
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.4))

# Fully Connected Layer 2
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(102, activation = 'softmax')) #As number of nodes in last layer in softmax is number of classes, where each node is probability of classes

In [6]:
# Compile the model
optimizer = SGD(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X_train, Y_train, batch_size=5, epochs=50, verbose=1, shuffle=True,validation_data=(X_test,Y_test))

Epoch 1/50
1372/1372 [==============================] - 82s 37ms/step - loss: 4.8831 - accuracy: 0.1516 - val_loss: 5.2424 - val_accuracy: 0.1495
Epoch 2/50
1372/1372 [==============================] - 49s 36ms/step - loss: 4.3274 - accuracy: 0.1951 - val_loss: 3.6343 - val_accuracy: 0.2641
Epoch 3/50
1372/1372 [==============================] - 49s 36ms/step - loss: 3.8445 - accuracy: 0.2437 - val_loss: 3.5499 - val_accuracy: 0.2899
Epoch 4/50
1372/1372 [==============================] - 49s 36ms/step - loss: 3.6032 - accuracy: 0.2779 - val_loss: 3.4449 - val_accuracy: 0.2921
Epoch 5/50
1372/1372 [==============================] - 49s 36ms/step - loss: 3.4243 - accuracy: 0.2976 - val_loss: 3.2272 - val_accuracy: 0.3052
Epoch 6/50
1372/1372 [==============================] - 49s 36ms/step - loss: 3.2501 - accuracy: 0.3205 - val_loss: 3.0051 - val_accuracy: 0.3472
Epoch 7/50
1372/1372 [==============================] - 49s 36ms/step - loss: 3.0550 - accuracy: 0.3510 - val_loss: 2.8107 -